## Importando base de dados Sinesp VDE 2023-2025 para PostgreSQL

Vamos trabalhar com as bases de dados do Validador de Dados Estatísticos do Sistema Nacional de Informações de Segurança Pública (Sinesp-VDE). Os dados têm um formato consolidado desde 2023, então vamos trabalhar com esses arquivos.

In [1]:
years = [2023, 2024, 2025]

### Importação

In [7]:
from pathlib import Path

import pandas as pd
from pandas import DataFrame as DF

from codetiming import Timer

dfs = []

for y in years:
    filename = f'BancoVDE {y}'
    csv_filename = Path(f'{filename}.csv')
    if csv_filename.exists():
        print(f'Dados Sinesp VDE de {y} disponíveis em formato CSV, carregando de {csv_filename}')
        with Timer(f'Carga de {csv_filename}'):
            df = pd.read_csv(csv_filename, low_memory=False) # low_memory evita warning the "mixed types"
    else:
        xlsx_filename = Path(f'{filename}.xlsx')
        print(f'Dados Sinesp VDE de {y} não disponíveis em formato CSV, carregando de {xlsx_filename}')
        with Timer(f'Carga de {xlsx_filename}'):
            df = pd.read_excel(xlsx_filename)
        print(f'Dados carregados de XLSX')
        print(f'Salvando base de {y} como CSV (porque CSVs são mais rápidos de carregar)')
        with Timer(f'Salvamento de {csv_filename}'):
            df.to_csv(csv_filename)
        print(f'Base de {y} salva como CSV em {csv_filename}')
    dfs.append(df)

Dados Sinesp VDE de 2023 disponíveis em formato CSV, carregando de BancoVDE 2023.csv
Elapsed time: 1.1375 seconds
Dados Sinesp VDE de 2024 disponíveis em formato CSV, carregando de BancoVDE 2024.csv
Elapsed time: 0.9994 seconds
Dados Sinesp VDE de 2025 disponíveis em formato CSV, carregando de BancoVDE 2025.csv
Elapsed time: 0.8206 seconds


Todos os dataframes têm as mesmas colunas:

In [19]:
columns = zip(*[df.columns for df in dfs])
DF({
  'year': years  
} | {
    f'c{i}' : [cs[y-years[0]] for y in years] for i, cs in enumerate(columns)
}).transpose()

,0,1,2
year,2023,2024,2025
c0,Unnamed: 0,Unnamed: 0,Unnamed: 0
c1,uf,uf,uf
c2,municipio,municipio,municipio
c3,evento,evento,evento
c4,data_referencia,data_referencia,data_referencia
c5,agente,agente,agente
c6,arma,arma,arma
c7,faixa_etaria,faixa_etaria,faixa_etaria
c8,feminino,feminino,feminino


Vejamos algumas linhas:

In [30]:
dfs[0].head()

,Unnamed: 0,uf,municipio,evento,data_referencia,agente,arma,faixa_etaria,feminino,masculino,nao_informado,total_vitima,total,total_peso,abrangencia,formulario
0,0,AC,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.366,Estadual,Formulário 5
1,1,AL,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.729,Estadual,Formulário 5
2,2,AM,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,294.635,Estadual,Formulário 5
3,3,AP,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.530,Estadual,Formulário 5
4,4,BA,NÃO INFORMADO,Apreensão de Cocaína,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.470,Estadual,Formulário 5


Todas têm o mesmo formato, aparentemente. Vamos concatenar os dataframes:

In [20]:
df = pd.concat(dfs)

In [ ]:
`Unnamed: 0` é um identificador, não nos interessa muito mas podemos preservar para referenciar de volta:

In [31]:
df.rename(columns={'Unnamed: 0': 'id_sinesp_vde'}, inplace=True)

Valor `total` não está claro, vejamos algumas linhas:

In [36]:
df[~df['total'].isna()].head()

,id_sinesp_vde,uf,municipio,evento,data_referencia,agente,arma,faixa_etaria,feminino,masculino,nao_informado,total_vitima,total,total_peso,abrangencia,formulario
648,648,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Carabina,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Estadual,Formulário 6
649,649,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Espingarda,NaN,NaN,NaN,NaN,NaN,5.0,NaN,Estadual,Formulário 6
650,650,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Fuzil,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Estadual,Formulário 6
651,651,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Metralhadora,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Estadual,Formulário 6
652,652,AC,NÃO INFORMADO,Arma de Fogo Apreendida,2023-01-01,NaN,Outra,NaN,NaN,NaN,NaN,NaN,16.0,NaN,Estadual,Formulário 6


Nos casos, se refere a armas de fogo. Haverá outros possíveis tipos de eventos?

In [37]:
df[~df['total'].isna()]['evento'].unique()

array(['Arma de Fogo Apreendida', 'Atendimento pré-hospitalar',
       'Busca e salvamento', 'Combate a incêndios',
       'Emissão de Alvarás de licença', 'Furto de veículo',
       'Mandado de prisão cumprido', 'Realização de vistorias',
       'Roubo a instituição financeira', 'Roubo de carga',
       'Roubo de veículo', 'Tráfico de drogas'], dtype=object)

### Definição de DDL

#### Tabela `AgregacaoEvento` (principal)

```sql
CREATE TABLE AgregacaoEvento (
    id                 INTEGER PRIMARY KEY,
    data_referencia    DATE,
    vitimas_femininas  INTEGER,
    vitimas_masculinas INTEGER,
    vitimas_nao_inform INTEGER,
    total_vitimas      INTEGER,
    total_objetos      INTEGER,
    total_peso         INTEGER,
    tipo_evento_id     INTEGER NOT NULL REFERENCES TipoEvento(id),
    uf_id              INTEGER NOT NULL REFERENCES UF(id),
    municipio_id       INTEGER NOT NULL REFERENCES Municipio(id),
    abrangencia_id     INTEGER NOT NULL REFERENCES Abrangencia(id),
    formulario_id      INTEGER NOT NULL REFERENCES Formulario(id),
    orgao_agente_id    INTEGER REFERENCES OrgaoAgente(id),
    arma_id            INTEGER REFERENCES Arma(id),
    faixa_etaria_id    INTEGER REFERENCES FaixaEtaria(id)
);
```

#### Tabela `TipoEvento`

Para essa tabela, precisamos saber o tamanho ideal da coluna com o tipo do evento:

In [83]:
def values_and_sizes(df, coluna):
    valores = df[coluna].unique()
    return DF({
        coluna: valores,
        'tamanho_nome': [len(v) if isinstance(v, str) else 0 for v in valores]
    })
    
eventos_df = values_and_sizes(df, 'evento')
eventos_df

,evento,tamanho_nome
0,Apreensão de Cocaína,20
1,Apreensão de Maconha,20
2,Arma de Fogo Apreendida,23
3,Atendimento pré-hospitalar,26
4,Busca e salvamento,18
5,Combate a incêndios,19
6,Emissão de Alvarás de licença,29
7,Estupro,7
8,Estupro de vulnerável,21
9,Feminicídio,11


Os nomes podem ser bem grandes e descritivos, temos um com 66 letras. Vamos usar 80 então.

O tipo de evento também define se há:
* contagem de vítimas (masculinas, femininas, desconhecidas, total);
* faixa etária
* peso de drogas apreendidas;
* contagem de "objetos" (desde armas apreendidas a mandatos executados);
* tipo de arma;

Isso será útil para categorizar os tipos de eventos, então vou gerar colunas nos eventos com esses dados:

```sql
CREATE TABLE TipoEvento (
    id                 INTEGER PRIMARY KEY,
    evento             VARCHAR(80) UNIQUE NOT NULL,
    tem_vitima         BOOLEAN NOT NULL,
    tem_faixa_etaria   BOOLEAN NOT NULL,
    tem_arma           BOOLEAN  NOT NULL,
    tem_peso           BOOLEAN NOT NULL,
    tem_objeto         BOOLEAN NOT NULL
);
```

#### Tabela `OrgaoAgente`

Vejamos os valores da coluna `agente`:

In [84]:
agentes_df = values_and_sizes(df, 'agente')
agentes_df

,agente,tamanho_nome
0,NaN,0
1,Agente de Trânsito,18
2,Bombeiro Militar,16
3,Guarda Municipal,16
4,Polícia Civil,13
5,Polícia Federal,15
6,Polícia Militar,15
7,Polícia Penal,13
8,Polícia Rodoviária Federal,26
9,Profissionais de Perícia,24


Uma coluna  com 30 caracteres parece suficiente:

```sql
CREATE TABLE OrgaoAgente (
    id            INTEGER PRIMARY KEY,
    orgao         VARCHAR(30) UNIQUE NOT NULL
);
```

#### Tabela `UF`

São as unidades da federação. Seus valores:

In [85]:
ufs_df = values_and_sizes(df, 'uf')
ufs_df

,uf,tamanho_nome
0,AC,2
1,AL,2
2,AM,2
3,AP,2
4,BA,2
5,CE,2
6,DF,2
7,ES,2
8,GO,2
9,MA,2


Além da sigla, vou querer também o nome inteiro. Vamos usar essa base de dados encontrada no site do Ministério do Desenvolvimento Social para ver o maior nome de estado:

In [88]:
estados_df = pd.read_csv(
    'Lista_Estados_Brasil_Versao_CSV.csv',
    encoding='ISO-8859-1', sep=';')
estados_df.head()

,IBGE,Estado,UF,Região,Qtd Mun,Sintaxe,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,11,Rondônia,RO,Região Norte,52,11'RO',NaN,NaN,PROCV(A2;'Lista de Estados IBGE'!$A$2:$C$28;2;0)
1,12,Acre,AC,Região Norte,22,12'AC',NaN,NaN,NaN
2,13,Amazonas,AM,Região Norte,62,13'AM',NaN,NaN,NaN
3,14,Roraima,RR,Região Norte,15,14'RR',NaN,NaN,NaN
4,15,Pará,PA,Região Norte,144,15'PA',NaN,NaN,NaN


In [90]:
estados_nomes_df = values_and_sizes(estados_df, 'Estado')
estados_nomes_df

,Estado,tamanho_nome
0,Rondônia,8
1,Acre,4
2,Amazonas,8
3,Roraima,7
4,Pará,4
5,Amapá,5
6,Tocantins,9
7,Maranhão,8
8,Piauí,5
9,Ceará,5


O maior nome tem 19 letras, então vamos pôr 25. Também é interessante usar a região, não havia pensando nisso! Vamos encolher o nome e avaliar

In [99]:
estados_df['Região'] = estados_df['Região'].str.replace('Região', '')
regioes_df = values_and_sizes(estados_df, 'Região')
regioes_df

,Região,tamanho_nome
0,Norte,6
1,Nordeste,9
2,Sudeste,8
3,Sul,4
4,Centro-Oeste,13


```sql
CREATE TABLE UF (
    id           INTEGER PRIMARY KEY,
    sigla        CHAR(2) NOT NULL,
    nome         VARCHAR(25) NOT NULL,
    regiao_id    INTEGER NOT NULL REFERENCES Regiao(id)
);

CREATE TABLE Regiao (
    id           INTEGER PRIMARY KEY,
    nome         VARCHAR(15),
);
```

### Tabela `Municipio`

Podemos pegar nomes dos municípios todos da base de dados de populações por ano do IBGE:

In [102]:
municipios_df = pd.read_excel('estimativa_dou_2025.ods', sheet_name='Municípios', header=1)
municipios_df

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA
0,RO,11.0,15.0,Alta Floresta D'Oeste,22787.0
1,RO,11.0,23.0,Ariquemes,109170.0
2,RO,11.0,31.0,Cabixi,5664.0
3,RO,11.0,49.0,Cacoal,98280.0
4,RO,11.0,56.0,Cerejeiras,16966.0
...,...,...,...,...,...
5568,GO,52.0,22203.0,Vila Boa,4145.0
5569,GO,52.0,22302.0,Vila Propício,6028.0
5570,DF,53.0,108.0,Brasília,2996899.0
5571,NaN,NaN,NaN,NaN,NaN


In [106]:
valores_coluna_df(municipios_df, 'NOME DO MUNICÍPIO').sort_values(by='tamanho_nome')

,NOME DO MUNICÍPIO,tamanho_nome
5298,NaN,0
2596,Luz,3
966,Ipu,3
3755,Uru,3
4591,Ipê,3
...,...,...
3139,São José do Vale do Rio Preto,29
1407,São Sebastião de Lagoa de Roça,30
2895,São Sebastião da Vargem Alegre,30
847,São Francisco de Assis do Piauí,31


O maior nome de município tem 32 letras, vamso usar 40:

```sql
CREATE_TABLE Municipio (
    id           INTEGER PRIMARY KEY,
    nome         VARCHAR(40) NOT NULL,
    estado_id    INTEGER REFERENCES Estado(id)
);
```
#### Tabela `arma` 


In [107]:
values_and_sizes(df, 'arma')

,arma,tamanho_nome
0,NaN,0
1,Carabina,8
2,Espingarda,10
3,Fuzil,5
4,Metralhadora,12
5,Outra,5
6,Pistola,7
7,Revolver,8
8,Rifle,5
9,Submetralhadora,15


```sql
CREATE TABLE Arma (
    id           INTEGER PRIMARY KEY,
    nome         VARCHAR(20)
})
```

#### Coluna `faixa_etaria`



In [108]:
values_and_sizes(df, 'faixa_etaria')

,faixa_etaria,tamanho_nome
0,NaN,0
1,Idade Não Informada,19
2,Maior de Idade,14
3,Menor de Idade,14


```sql
CREATE TABLE FaixaEtaria (
    id          INTEGER PRIMARY KEY,
    faixa       VARCHAR(25)
);
```

### Inserções de valores derivativos (DDM)

Vamos ver como como esses se relacionam:

In [68]:
victim_columns = [
    'feminino', 'masculino', 'nao_informado', 'total_vitima'
]
event_related_columns = victim_columns + [
    'arma', 'faixa_etaria', 
    'total_peso', 'total'
]

event_column_df = df.groupby('evento')[event_related_columns].agg(
    lambda c: c.notna().any()
)
event_column_df['vitima'] =  event_column_df[victim_columns].any(axis=1)
event_column_df.drop(victim_columns, axis=1, inplace=True)
event_column_df


,arma,faixa_etaria,total_peso,total,vitima
evento,,,,,
Apreensão de Cocaína,False,False,True,False,False
Apreensão de Maconha,False,False,True,False,False
Arma de Fogo Apreendida,True,False,False,True,False
Atendimento pré-hospitalar,False,False,False,True,False
Busca e salvamento,False,False,False,True,False
Combate a incêndios,False,False,False,True,False
Emissão de Alvarás de licença,False,False,False,True,False
Estupro,False,False,False,False,True
Estupro de vulnerável,False,False,False,False,True


### A fazer

- [ ] Nomear constrints de chave estrangeira no PostgreSQL?

#### SANDBOX TABELAS
CREATE TABLE TipoEvento (
    id        INTEGER PRIMARY KEY,
);
    CREATE TABLE UF (
    id        INTEGER PRIMARY KEY,
);
CREATE TABLE Municipio (
    id        INTEGER PRIMARY KEY,
);
CREATE TABLE Abrangencia (
    id        INTEGER PRIMARY KEY,
);
CREATE TABLE Formulario (
    id        INTEGER PRIMARY KEY,
);
CREATE TABLE OrgaoAgente (
    id        INTEGER PRIMARY KEY,
);
CREATE TABLE Arma (
    id        INTEGER PRIMARY KEY,
);
CREATE TABLE FaixaEtaria (
    id        INTEGER PRIMARY KEY,
);